# Fundamental rythms analysis in SSVEPs as a quantifying measure for visual fatigue 

## Declarando bibliotecas 

In [ ]:
%config Completer.use_jedi = False

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import loadmat
from scipy.fft import fft, fftshift, fftfreq
from nested_dict import nested_dict
from mpl_toolkits import mplot3d

## Funções

In [ ]:
def createDataframe(filePath,channelQnt):

    # Carregando o arquivo .m
    matFile = loadmat(filePath)['ans']

    # Lista de dicionários
    signalsFreq = [{

        # Frequências de estimulação
        'freq': matFile[:,stimFreqIdx][0][0][0][1][0][0],

        # Lista de listas, em que cada uma representa um sinal de um eletrodo, de acordo com a frequência de estímulo
        'signals': [[np.array(matFile[:,stimFreqIdx][0][0][0][0][:,channelIdx][Fs*2:Fs*12])] for channelIdx in range(channelQnt)]

    # Um dicionário para cada frequência de estimulação
    } for stimFreqIdx in range(matFile.shape[1])]

    # Transformando a lista de dicionário em um dataframe Pandas. Selecionando as frequências de estímulo como índices do dataframe
    signalsDF = pd.DataFrame(signalsFreq).set_index('freq')

    # Em determinadas faixas de frequência o passo de frequência não é de 0,5 Hz. Aqui são removidos os valores de frequência que não são divisíveis por 0,5
    for freq in signalsDF.index:
        if freq%0.5 != 0:
            signalsDF.drop(freq, inplace=True)

    return signalsDF

In [ ]:
def createResultsDataframe(numWindows):
    

In [ ]:
def preProcessing(signalsDF):
    
    for( )

## Declaração de variáveis

In [ ]:
filePath = '/home/guilherme/pCloudDrive/myFolder/estudos/ifes/pesquisaSSVEP/base_dados/dados_ordenados/Voluntários Argentina/Baixa Frequência/Vol_1_LOW.mat'

# Duração do estímulo em segundos
stimPeriod = 10

# Número de canais (eletrodos)
channelQnt = 12

# Frequência amostral em Hz
Fs = 512

# Número de pontos vizinhos à frequência desejada para calculo do SNR
neighboorSNR = 4

# Tamanho das janelas temporais em segundos
windowSize = 2 

# Número de janelas temporais
numWindows = 9

# Número de amostras por janela
nFFT = windowSize*Fs

# Passo de frequência para plot, em Hz
deltaF = Fs/nFFT

# Dicionário com intervalos dos ritmos fundamentais, em passos de 0.5 Hz
fundRythms = {
    'delta': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0],
    'theta': [4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5],
    'alpha1': [8.0, 8.5, 9.0, 9.5, 10.0],
    'alpha2': [10.5, 11.0, 11.5, 12.0, 12.5, 13.0],
    'beta': [14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 25.5, 26.0, 26.5, 27.0, 27.5, 28.0, 28.5, 29.0, 29.5, 30.0]
}

# Banda de passagem do filtro elíptico passa-banda
bandWidth = np.array([3,90])/(Fs/2)

# Declaração do filtro ellíptico
ellipNum, ellipDen = signal.ellip(5, 1, 15, bandWidth, 'bandpass')

# Definição da frequência a ser aplicado o filtro notch
notchFreq = 50

# Ftor de qualidade do filtro notch
qltyFactor = 35

# Declaração do filtro notch
notchNum, notchDen = signal.iirnotch(notchFreq, qltyFactor, Fs)